# Levantamento Cortiços e ZEIS-3 - SEHAB

## Requisitos:

Boa tarde!!
Encaminho uma tabela com o cruzamento entre os imóveis originados do arquivo “ vistoria prévia” classificados como cortiços e ocupações, somado as ZEIS 3 que foram inseridas, com a base de lotes fiscal. Identificamos nessa tabela o código do cadastro imobiliário dos imóveis (SQL).
Solicitamos a gentileza da equipe da SEHAB, para que faça gestão junto a Secretaria de Finanças, para que disponibilizem os atributos do cadastro imobiliário desses imóveis como:

Dados dos proprietários
Dados do terreno (área, ano de lançamento do lote, posição e testada)
Dados da construção (área, tipo, material, padrão, utilização), valor venal do imóvel e histórico da indicação fiscal.
Informações sobre tombamentos.

Salientamos que esses dados serão fundamentais para o preenchimento da ficha de caracterização física dos imóveis.

Outra solicitação à equipe da SEHAB, é que nos deem a diretriz para a inserção de código chave nos imóveis inseridos através da ZEIS 3 seguindo os padrões do Habitasampa.
Segue a tabela.


## Considerações técnicas:

Alguns dos dados solicitados (identificação dos proprietários) são dados pessoais protegidos pela LGPD que não estão disponíveis publicamente.

Estes deverão ser solicitados diretamente à SF pela SEHAB.

Os demais, acrescentamos aqui neste script.

In [13]:
import os
import pandas as pd
from requests import Session
import zipfile

In [11]:
def make_data_dir(folder='data'):
    
    if not os.path.exists(folder):
        os.mkdir(folder)
        
    return os.path.abspath(folder)

def write_file(fname, content, file_type='t', folder = None):
    
    if folder is not None:
        folder = make_data_dir(folder)
        
        fname = os.path.join(folder, fname)
        
    with open(fname, f'w{file_type}') as f:
        f.write(content)
    
    
    return fname


def download_binary_data(session, url, fname, folder='data'):
    
    with session.get(url) as r:
        assert r.status_code==200
        content = r.content
    
    write_file(fname, content, file_type='b', folder=folder)
    
    
def download_iptu(url=None, fname='iptu.zip', folder='data'):
    
    if url is None:
        url = ("http://download.geosampa.prefeitura.sp.gov.br/PaginasPublicas/"
               "downloadArquivo.aspx?orig=DownloadCamadas&arq=12_Cadastro%5CIPTU_INTER%5CXLS_CSV%5CIPTU_2022&arqTipo=XLS_CSV")
        
    session = Session()
    session.get('http://geosampa.prefeitura.sp.gov.br/PaginasPublicas')
    session.headers = {'Accept-Encoding' : 'gzip, deflate',
                       'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
                      'Referer' : 'http://geosampa.prefeitura.sp.gov.br/PaginasPublicas/_SBC.aspx'}
    
    
    download_binary_data(session, url, fname, folder)
    

In [12]:
download_iptu()

In [14]:
def unzip_iptu(file='iptu.zip', folder='data'):
    
    if folder:
        file = os.path.abspath(os.path.join(folder, file))
    
    with zipfile.ZipFile(file, 'r') as zip_ref:
        zip_ref.extractall(folder)


In [15]:
unzip_iptu()

In [17]:
df_iptu = pd.read_csv(os.path.join('data', 'iptu.zip'), encoding='latin-1', sep=';')

In [18]:
df_iptu.head()

,NUMERO DO CONTRIBUINTE,ANO DO EXERCICIO,NUMERO DA NL,DATA DO CADASTRAMENTO,NUMERO DO CONDOMINIO,CODLOG DO IMOVEL,NOME DE LOGRADOURO DO IMOVEL,NUMERO DO IMOVEL,COMPLEMENTO DO IMOVEL,BAIRRO DO IMOVEL,...,ANO DA CONSTRUCAO CORRIGIDO,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,TIPO DE TERRENO,FATOR DE OBSOLESCENCIA,ANO DE INICIO DA VIDA DO CONTRIBUINTE,MES DE INICIO DA VIDA DO CONTRIBUINTE,FASE DO CONTRIBUINTE
0,0010030001-4,2022,1,01/01/22,00-0,03812-1,R S CAETANO,13.0,NaN,SANTA EFIGENIA,...,1924,1,"13,00",Loja,Comercial horizontal - padrão B,De esquina,"0,20",1963,1,0
1,0010030002-2,2022,1,01/01/22,00-0,03812-1,R S CAETANO,19.0,NaN,SANTA EFIGENIA,...,1944,1,"6,00",Loja,Comercial horizontal - padrão B,Normal,"0,20",1963,1,0
2,0010030003-0,2022,1,01/01/22,00-0,03812-1,R S CAETANO,27.0,NaN,SANTA EFIGENIA,...,1965,2,"7,85",Loja,Comercial horizontal - padrão B,Normal,"0,26",1963,1,0
3,0010030004-9,2022,1,01/01/22,00-0,03812-1,R S CAETANO,33.0,NaN,NaN,...,1944,1,"6,05",Loja,Comercial horizontal - padrão B,Normal,"0,20",1963,1,0
4,0010030005-7,2022,1,01/01/22,00-0,03812-1,R S CAETANO,39.0,NaN,NaN,...,2004,2,"6,70",Loja,Comercial horizontal - padrão B,Normal,"0,84",1963,1,0


In [22]:
df_sehab = pd.read_excel(os.path.join('data', 'VistoriaXIPTU.xlsx'), dtype='object')

In [23]:
df_sehab.head()

,uuid_id,Endereço,Distrito,Classificação (Mapa),ID Lote (IPTU),Setor (IPTU),Quadra (IPTU),Lote (IPTU),Condominio (IPTU)
0,f952bdc5-0315-44c7-8c79-5d4bca9efcc5,Rua dos Estudantes 566,SE,cortiços,701,005,057,0004,00
1,f952bdc5-0315-44c7-8c79-5d4bca9efcc5,Rua dos Estudantes 566,SE,cortiços,1495,005,057,0015,00
2,f952bdc5-0315-44c7-8c79-5d4bca9efcc5,Rua dos Estudantes 566,SE,cortiços,2060,005,057,0016,00
3,e1df1cf7-4222-40a3-a17b-8da0bba72036,Rua Conselheiro Ramalho 466,BELA VISTA,cortiços,19841,009,006,0010,00
4,5a587a36-798b-4518-bcc9-2cbadad1e289,Rua Abolição 370,REPUBLICA,cortiços,4862,006,049,0004,00


In [28]:
df_sehab['Condominio (IPTU)'].unique()

array(['00', '09', '04', '10', '02', '07', '01', '11', '05', '12', '03',
       '19', '06', '08', '18', '13'], dtype=object)

In [29]:
def build_sql(row):
    
    setor = row['Setor (IPTU)']
    quadra = row['Quadra (IPTU)']
    lote = row['Lote (IPTU)']
    #condominio veio com dois digitos, queremos só o ultimo
    condominio = row['Condominio (IPTU)'][-1]
    
    return f'{setor}{quadra}{lote}-{condominio}'

In [31]:
df_sehab['NUMERO DO CONTRIBUINTE'] = df_sehab.apply(lambda row: build_sql(row), axis=1)

In [32]:
df_sehab['NUMERO DO CONTRIBUINTE'].apply(len).unique()

array([12])

In [33]:
df_iptu['NUMERO DO CONTRIBUINTE'].apply(len).unique()

array([12])

In [35]:
sehab_iptu = pd.merge(df_sehab, df_iptu, how='left', on = 'NUMERO DO CONTRIBUINTE')

In [36]:
sehab_iptu.to_csv(os.path.join('data', 'sehab_iptu.csv'), sep=';', encoding='utf-8', index=False)

In [40]:
teste_lote_sem_cond = '0330050000'

In [41]:
df_iptu[df_iptu['NUMERO DO CONTRIBUINTE'].apply(lambda x: '0030140204' in x)]

,NUMERO DO CONTRIBUINTE,ANO DO EXERCICIO,NUMERO DA NL,DATA DO CADASTRAMENTO,NUMERO DO CONDOMINIO,CODLOG DO IMOVEL,NOME DE LOGRADOURO DO IMOVEL,NUMERO DO IMOVEL,COMPLEMENTO DO IMOVEL,BAIRRO DO IMOVEL,...,ANO DA CONSTRUCAO CORRIGIDO,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,TIPO DE TERRENO,FATOR DE OBSOLESCENCIA,ANO DE INICIO DA VIDA DO CONTRIBUINTE,MES DE INICIO DA VIDA DO CONTRIBUINTE,FASE DO CONTRIBUINTE
27555,0030140204-3,2022,1,01/01/22,00-0,13796-0,R MELO BARRETO,104.0,E 110,BRAS,...,1964,3,"10,00","Prédio de apartamento, não em condomínio, de u...",Residencial vertical - padrão C,Normal,"0,24",1963,1,0


In [43]:
df_sehab['SQL_SEM_COND'] = df_sehab['NUMERO DO CONTRIBUINTE'].apply(lambda x: x[:-2])

In [44]:
df_iptu['SQL_SEM_COND'] = df_iptu['NUMERO DO CONTRIBUINTE'].apply(lambda x: x[:-2])

In [46]:
sehab_iptu_sem_cond = pd.merge(df_sehab, df_iptu, how='left', on = 'SQL_SEM_COND')

In [48]:
sehab_iptu['ANO DO EXERCICIO'].isnull().mean()

0.9148797920727745

In [49]:
sehab_iptu_sem_cond['ANO DO EXERCICIO'].isnull().mean()

0.11760883690708251

In [50]:
sehab_iptu_sem_cond.to_csv(os.path.join('data', 'sehab_iptu_sem_cond.csv'), sep=';', encoding='utf-8', index=False)